# Arrays and views

Julia has excellent functionality for manipulating arrays and for linear algebra. We will have a quick look at this subject, which is much more complicated than you might suspect; see e.g. the talk on "Taking vector transposes seriously".

Let's define a 2x2 array (matrix):

In [27]:
M = [1 2 3; 4 5 6; 7 8 9]  # a 3x3 matrix

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

In [28]:
typeof(M)

Array{Int64,2}

We can extract part of the matrix using indexing notation:

In [29]:
part = M[2:3, 1:2]

2×2 Array{Int64,2}:
 4  5
 7  8

What happens if we modify `part`?

In [30]:
part[1, 1]

4

In [31]:
part[1, 1] = 10

10

In [32]:
part

2×2 Array{Int64,2}:
 10  5
  7  8

In [33]:
M

3×3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

We see that `M` has *not* been modified: `part` was a **copy** of that part of `M`.

## Views

We often do *not* want a copy, but rather just a reference to the same data, which is called a `view`: 

In [34]:
V = view(M, 2:3, 1:2)

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 4  5
 7  8

In [35]:
typeof(V)

SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}

Although this type looks complicated, it just contains the necessary information for the object to manipulate correctly the underlying data.

If we modify `V`, then `M` also gets modified, since it is the same data:

In [36]:
V[1, 1]

4

In [37]:
V[1, 1] = 100

100

In [38]:
V

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 100  5
   7  8

In [39]:
M

3×3 Array{Int64,2}:
   1  2  3
 100  5  6
   7  8  9

We can also write

In [40]:
@view M[2:3, 1:2]

2×2 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 100  5
   7  8

for ease of use.

## In-place and vectorized operations: "`.`" ("pointwise")

Suppose we have two matrices and wish to add one to the other:

In [29]:
A = rand(1000, 1000)
B = rand(1000, 1000);

Coming from other languages, we might expect to write `A += B`, and indeed this works:

In [11]:
A += B

1000×1000 Array{Float64,2}:
 1.38641   0.314192   1.53606   1.13381   …  1.05444   0.909165  0.64833  
 0.848104  1.62689    1.01003   1.05422      1.20477   0.645419  0.0310397
 1.60988   0.170039   0.623727  1.0248       1.11513   0.437426  0.531131 
 0.937548  0.275432   0.622256  1.82359      1.13788   1.08481   1.6772   
 1.24      1.14651    1.30733   1.15012      1.44183   0.668125  1.52232  
 1.26722   1.5369     0.93767   0.888896  …  1.26697   1.12054   1.93831  
 0.774109  0.659617   0.867579  0.949677     0.754672  0.813245  0.961856 
 1.1071    0.621759   1.87547   0.638792     1.04591   0.606587  0.902166 
 0.736286  1.20555    1.06171   0.428142     1.56834   1.52938   0.887196 
 0.9663    1.44038    0.969669  0.525683     1.12883   1.03449   0.88939  
 0.549136  0.0727231  0.660895  0.923914  …  1.15862   1.66588   0.731828 
 0.86727   1.02377    1.00417   0.434366     1.52025   1.17064   0.530294 
 1.32514   1.32851    1.01702   1.54345      0.604151  1.26558   0.48657

This is just "syntactic sugar" (i.e. a cute way of writing) `A = A + B`.

However, it turns out that this does not do what you think it does, namely "in-place addition", in which each element of `A` is updated in place. Rather, it allocates a new temporary object for the result of `A + B`. We can see this:

In [8]:
using BenchmarkTools, Compat

In [39]:
@time A += B

  0.002885 seconds (6 allocations: 7.630 MiB)


1000×1000 Array{Float64,2}:
 18300.9   11573.4     9496.08   11073.7    …   1774.75   9014.89    8764.21 
  7645.83   5457.2     6135.05    9996.26        330.64  10813.7    18108.8  
  3605.05  15757.0     5983.59    3740.62      17787.2   10605.5    14001.3  
  8223.97  15365.9      230.393   9267.63      10822.2   16846.1     3765.78 
  9198.39  19170.7    15055.5    15482.8       13475.5    1797.14    8042.69 
  3392.75  17445.5     6254.16   10151.3    …  19300.8   13642.3     8335.98 
 14297.0    3297.45    7367.07    9915.52       2772.94  10958.9    11261.3  
 14409.6   11982.5     4521.68     784.552     13507.7    6124.94     729.804
  2359.62   9718.2      786.653  16736.9        6785.35  20503.7     9164.59 
  7179.09   9383.98   14786.6     8096.51       8453.89   7796.07     275.502
  2120.48    612.727  17590.8     4273.94   …  14357.1   18535.3    14461.0  
 12227.5   19725.7    16908.2    11577.7        9762.74   2495.77    6428.86 
 19733.4    8872.2     6461.5    191

Note the large amount of allocation here (1,000,000 $\times$ 8 bytes)

The in-place behaviour can be obtained using **pointwise operators**:

In [36]:
A .= A .+ B

  0.001119 seconds (8 allocations: 288 bytes)


1000×1000 Array{Float64,2}:
 18298.2   11571.7     9494.7    …   1774.49    9013.57    8762.93 
  7644.71   5456.4     6134.15        330.592  10812.1    18106.1  
  3604.53  15754.7     5982.72      17784.6    10603.9    13999.3  
  8222.77  15363.7      230.36      10820.6    16843.6     3765.23 
  9197.05  19167.9    15053.3       13473.6     1796.88    8041.52 
  3392.26  17443.0     6253.24   …  19298.0    13640.3     8334.76 
 14294.9    3296.97    7366.0        2772.54   10957.3    11259.7  
 14407.5   11980.7     4521.02      13505.8     6124.05     729.698
  2359.27   9716.78     786.538      6784.36   20500.7     9163.25 
  7178.04   9382.61   14784.4        8452.66    7794.94     275.462
  2120.17    612.637  17588.3    …  14355.0    18532.6    14458.9  
 12225.7   19722.8    16905.8        9761.32    2495.4     6427.92 
 19730.5    8870.91    6460.55      15439.1     3860.92    1211.52 
     ⋮                           ⋱                                 
 13847.4   13094.4  

In [37]:
@time A .= A .+ B

  0.001602 seconds (8 allocations: 288 bytes)


1000×1000 Array{Float64,2}:
 18299.1   11572.3     9495.16   …   1774.58    9014.01    8763.36 
  7645.09   5456.67    6134.45        330.608  10812.6    18107.0  
  3604.7   15755.5     5983.01      17785.5    10604.4    13999.9  
  8223.17  15364.5      230.371     10821.2    16844.4     3765.41 
  9197.5   19168.8    15054.1       13474.2     1796.96    8041.91 
  3392.42  17443.8     6253.55   …  19298.9    13641.0     8335.17 
 14295.6    3297.13    7366.36       2772.67   10957.8    11260.2  
 14408.2   11981.3     4521.24      13506.4     6124.34     729.733
  2359.39   9717.25     786.576      6784.69   20501.7     9163.69 
  7178.39   9383.07   14785.1        8453.07    7795.32     275.475
  2120.28    612.667  17589.1    …  14355.7    18533.5    14459.6  
 12226.3   19723.8    16906.6        9761.79    2495.53    6428.23 
 19731.5    8871.34    6460.87      15439.9     3861.1     1211.58 
     ⋮                           ⋱                                 
 13848.1   13095.1  

Furthermore, we can chain such operations together with no creation of temporaries:

In [46]:
C = rand(1000, 1000)

@btime A .+= B + C  # allocates

  2.626 ms (6 allocations: 7.63 MiB)


1000×1000 Array{Float64,2}:
  6871.03    7508.43   6151.07  10138.4    …   5878.45    4342.9   12620.3 
  2263.17    5360.1   11577.1    2205.83       6958.22    9295.74  11218.5 
   483.878   1856.49   3755.18  14813.2         115.145   2609.76  11516.7 
 12125.8    14842.5    3490.5   15315.2        1769.31    5973.3    2211.09
  9887.45   10989.5    9158.5   15452.5       12181.2    14525.2    6659.89
  4005.47   10119.7    7320.63   2867.35   …   4612.68   10629.1    2402.8 
 15877.0     7978.97  11259.8    5609.87       7807.92   12885.3   10213.0 
  3712.99    5512.19   9497.4   11834.9        5358.9     8356.29   2570.53
  4808.12   13984.4    2802.51  10297.9        7302.3    12307.5   11400.4 
  8550.36    9887.78  11272.6    7755.13       2589.72    7357.38  13315.8 
 11759.0     5040.06  13658.2    9208.91   …   1415.75    9477.93   6824.76
  6539.19    5129.52   4796.1    2844.78      15438.9     5459.98  10899.8 
 11661.3     8025.83  12628.0    2743.05       7688.21    35

In [47]:
@btime A .+= B .+ C  # does not allocate

  1.128 ms (4 allocations: 160 bytes)


1000×1000 Array{Float64,2}:
 14146.5   13674.5   15500.6   21106.7   …  11808.8    11325.8   20361.1 
  8400.02   8328.81  20612.0    4522.03     10871.8    14623.4   20382.8 
  1719.99   4333.43   8028.48  26482.2        238.767   4556.97  19861.2 
 21694.6   26409.9    9644.93  27830.7       6651.68   14901.1    5488.46
 17831.1   21956.4   17486.4   27811.0      22759.5    24967.1   15084.0 
  9838.66  19514.7   11734.7   10462.1   …   9127.62   19148.9    4762.53
 29448.8   14519.0   20407.8    9272.27     15940.1    24874.6   16150.0 
  5748.72  10318.7   19741.1   22521.2      12381.5    14114.1    5887.41
 13392.6   27275.9    9777.21  20438.6      11704.6    23703.1   20036.2 
 14882.3   19368.3   21752.3   16320.5       8339.06   12431.3   21157.8 
 19892.8   10699.7   26911.5   14695.3   …   2492.12   16109.2   12120.2 
 10952.5   10349.7    8455.05   6170.75     27977.8    12389.6   19107.6 
 22405.8   13025.1   24634.4    4520.95     14354.6     8298.81  24256.0 
     ⋮    

In [40]:
myfunc(x,y) = x + 2y

myfunc (generic function with 1 method)

In [41]:
myfunc.([1,2], [3,5])

2-element Array{Int64,1}:
  7
 12

See [this blog post by Steven Johnson](https://julialang.org/blog/2017/01/moredots) for more details.